선형 공간 필터링 : 회선을 이용한 영상처리 방법

비선형 공간 필터링 :

##### 7.3.1 최대값/최소값 필터링
최대값 필터링 : 마스크 계수 중 최대값을 통과시켜 출력 화소가 됨
- 돌출되는 어두운 값이 제거, 전체적으로 밝은 영상이 됨
- 밝은 임펄스 잡음이 강조

최소값 필터링 : 마스크 계수 중 최소값을 통과시켜 출력 화소가 됨
- 돌출되는 밝은 값이 제거, 전체적으로 어두운 영상이 됨
- 어두운 임펄스 잡음이 강조

In [1]:
# 7.3.1 최소값-최대값 필터링
import numpy as np, cv2

def minmax_filter(image, ksize, mode):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.uint8)
    center = ksize // 2

    for i in range(center, rows - center):
        for j in range(center, cols - center):
            y1, y2 = i - center, i + center + 1
            x1, x2 = j - center, j + center + 1
            mask = image[y1:y2, x1:x2]
            dst[i, j] = cv2.minMaxLoc(mask)[mode]
    return dst

image = cv2.imread("images_07/min_max.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

minfilter_img = minmax_filter(image, 3, 0)
maxfilter_img = minmax_filter(image, 3, 1)

cv2.imshow("image", image)
cv2.imshow("minfilter_img", minfilter_img)
cv2.imshow("maxfilter_img", maxfilter_img)
cv2.waitKey(0)

-1

In [2]:
# 7.3.2 평균값 필터링
import numpy as np, cv2

def average_filter(image, ksize):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.uint8)
    center = ksize // 2

    for i in range(center, rows - center):
        for j in range(center, cols - center):
            y1, y2 = i - center, i + center + 1
            x1, x2 = j - center, j + center + 1

            if y1 < 0 or y2 > rows or x1 < 0 or x2 > cols: # 입력 범위값을 벗어난 경우
                dst[i, j] = image[i, j]
            else:
                mask = image[y1:y2, x1:x2]
                dst[i, j] = cv2.mean(mask)[0]
    return dst

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

avg_img = average_filter(image, 5)
blur_img = cv2.blur(image, (5,5), (-1, -1), cv2.BORDER_REFLECT)
box_img = cv2.boxFilter(image, ddepth=-1, ksize=(5,5))

cv2.imshow("image", image)
cv2.imshow("avg_img", avg_img)
cv2.imshow("blur_img", blur_img)
cv2.imshow("box_img", box_img)
cv2.waitKey(0)

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'blur'
> Overload resolution failed:
>  - Can't parse 'anchor'. Input argument doesn't provide sequence protocol
>  - Can't parse 'anchor'. Input argument doesn't provide sequence protocol


In [3]:
# 7.3.3 미디안 필터링
import numpy as np, cv2

def median_filter(image, ksize):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.uint8)
    center = ksize // 2

    for i in range(center, rows - center):
        for j in range(center, cols - center):
            y1, y2 = i - center, i + center + 1
            x1, x2 = j - center, j + center + 1
            mask = image[y1:y2, x1:x2].flatten()

            sort_mask = cv2.sort(mask, cv2.SORT_EVERY_COLUMN)
            dst[i, j] = sort_mask[sort_mask.size//2]
    return dst

# 이미지에 강제로 노이즈를 만드는 함수
def salt_pepper_noise(img, n):
    h, w = img.shape[:2]
    x, y = np.random.randint(0, w, n), np.random.randint(0, h, n)
    noise = img.copy()
    for (x, y) in zip(x, y):
        noise[y, x] = 0 if np.random.rand() < 0.5 else 255
    return noise

image = cv2.imread("images_07/median2.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

noise = salt_pepper_noise(image, 500) # 노이즈 이미지 생성
med_img1 = median_filter(noise, 5)
med_img2 = cv2.medianBlur(noise, 5)

cv2.imshow("image", image)
cv2.imshow("noise", noise)
cv2.imshow("median", med_img1)
cv2.imshow("CV median", med_img2)
cv2.waitKey(0)

-1

##### 가우시안 필터링

정규 분포를 사용, 거리 대비로 나와 가까운 요소의 가중치를 높임

멀리 있는 요소들까지 가져오더라도(고려하더라도) 이미지 손상이 다른 필터링에 비해서 덜함

In [5]:
# 7.3.4 가우시안 필터링
import numpy as np, cv2

def getGaussianMask(ksize, sigmaX, sigmaY):
    sigma = 0.3 * ((np.array(ksize) - 1.0) * 0.5 - 1.0) + 0.8
    if sigmaX <= 0: sigmaX = sigma[0]
    if sigmaY <= 0: sigmaY = sigma[1]

    u = np.array(ksize)//2
    x = np.arange(-u[0], u[0]+1, 1)
    y = np.arange(-u[1], u[1]+1, 1)
    x, y = np.meshgrid(x, y)

    ratio = 1 / (sigmaX * sigmaX * 2 * np.pi)
    v1 = x ** 2 / (2 * sigmaX ** 2)
    v2 = y ** 2 / (2 * sigmaY ** 2)
    mask = ratio * np.exp(-(v1+v2))
    return mask / np.sum(mask)

image = cv2.imread("images_07/smoothing.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

ksize = (17, 5)
gaussian_2d = getGaussianMask(ksize, 0, 0)
gaussian_1dX = cv2.getGaussianKernel(ksize[0], 0, cv2.CV_32F)
gaussian_1dY = cv2.getGaussianKernel(ksize[1], 0, cv2.CV_32F)

gauss_img1 = cv2.filter2D(image, -1, gaussian_2d)
gauss_img2 = cv2.GaussianBlur(image, ksize, 0)
gauss_img3 = cv2.sepFilter2D(image, -1, gaussian_1dX, gaussian_1dY)

titles = ['image', 'gauss_img1', 'gauss_img2', 'gauss_img3']
for t in titles: cv2.imshow(t, eval(t))
cv2.waitKey(0)

27

In [15]:
# 7.3.5 블러링과 캐니 엣지를 이용한 컬러 엣지 검출
import cv2

def onTrackbar(th):
    rep_edge = cv2.GaussianBlur(rep_gray, (5, 5), 0)
    rep_edge = cv2.Canny(rep_edge, th, th*2, 5)
    h, w = image.shape[:2]
    cv2.rectangle(rep_edge, (0, 0, w, h), 255, -1)
    color_edge = cv2.bitwise_and(rep_image, rep_image, mask=rep_edge)
    cv2.imshow("color edge", color_edge)

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_COLOR)
if image is None: raise Exception("영상 파일 읽기 오류")

th = 50
rep_image = cv2.repeat(image, 1, 2)
rep_gray = cv2.cvtColor(rep_image, cv2.COLOR_BGR2GRAY)

cv2.namedWindow("color edge", cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar("Canny th", "color edge", th, 100, onTrackbar)
onTrackbar(th)
cv2.waitKey(0)

-1